In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
import datetime

In [ ]:
CHERNOBYL_COORDS = (30.0927, 51.3870) # (Lon, lat) or (x, y)

MAP_SIZE_IN_DEGREES = (100, 50)

LOWER_LEFT_X_LON = CHERNOBYL_COORDS[0] - MAP_SIZE_IN_DEGREES[0]/2
LOWER_LEFT_Y_LAT = CHERNOBYL_COORDS[1] - MAP_SIZE_IN_DEGREES[1]/2
UPPER_LEFT_X_LON = CHERNOBYL_COORDS[0] + MAP_SIZE_IN_DEGREES[0]/2
UPPER_LEFT_Y_LAT = CHERNOBYL_COORDS[1] + MAP_SIZE_IN_DEGREES[1]/2



# Display Chernobyl Location

In [ ]:
fig = plt.figure(figsize=(12,9))

# world map has a lower left corner of (-180, -90) and an upper right corner of (180, 90)
m = Basemap(projection='mill',
            llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
            llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
            urcrnrlon=UPPER_LEFT_X_LON, 
            urcrnrlat=UPPER_LEFT_Y_LAT,
            resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

m.shadedrelief() # makes it look pretty :)
m.drawcountries(linewidth=1.2)

# Locate Chernobyl
m.scatter(CHERNOBYL_COORDS[0], CHERNOBYL_COORDS[1], latlon=True, s=100, c='green', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

plt.show()

# Get Sensor Locations

In [ ]:
sensor_latitudes = []
sensor_longitudes = []

data = pd.read_csv("data/cleaned.csv")

for lon, lat in data.filter(['X','Y']).drop_duplicates().values:
    # print(lon, lat)
    sensor_longitudes.append(lon)
    sensor_latitudes.append(lat)


assert len(sensor_latitudes) == len(sensor_longitudes) and len(sensor_latitudes) > 50

# Display Sensor Locations

In [ ]:
fig = plt.figure(figsize=(12,9))

# world map has a lower left corner of (-180, -90) and an upper right corner of (180, 90)
m = Basemap(projection='mill',
            llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
            llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
            urcrnrlon=UPPER_LEFT_X_LON, 
            urcrnrlat=UPPER_LEFT_Y_LAT,
            resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

m.shadedrelief() # makes it look pretty :)
m.drawcountries(linewidth=1.2)
m.drawrivers(color="lightblue")

# Locate Chernobyl
m.scatter(CHERNOBYL_COORDS[0], CHERNOBYL_COORDS[1], latlon=True, s=100, c='green', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

#Locate Each Sensor:
m.scatter(sensor_longitudes, sensor_latitudes, latlon=True, s=50, c='blue', marker='o', alpha=1) 



plt.show()

# Interactive Map

In [ ]:
colorbar = {
    'I_131_(Bq/m3)':{
        'min':0,
        'max':0
    },
    'Cs_134_(Bq/m3)':{
        'min':0,
        'max':0
    },
    'Cs_137_(Bq/m3)':{
        'min':0,
        'max':0
    },
}
v=[]
for lon, lat, value in data.filter(['X','Y', "I_131_(Bq/m3)"]).drop_duplicates().values:
    v.append(value)
colorbar['I_131_(Bq/m3)']['min'] = min(v)
colorbar['I_131_(Bq/m3)']['max'] = max(v)

v=[]
for lon, lat, value in data.filter(['X','Y', "Cs_134_(Bq/m3)"]).drop_duplicates().values:
    v.append(value)
colorbar['Cs_134_(Bq/m3)']['min'] = min(v)
colorbar['Cs_134_(Bq/m3)']['max'] = max(v)

v=[]
for lon, lat, value in data.filter(['X','Y', "Cs_137_(Bq/m3)"]).drop_duplicates().values:
    v.append(value)
colorbar['Cs_137_(Bq/m3)']['min'] = min(v)
colorbar['Cs_137_(Bq/m3)']['max'] = max(v)

In [ ]:
def create_map(date, iso_widget, CREATE_PICS=False):
    sensor_latitudes = []
    sensor_longitudes = []
    iso_values = []

    if date.day >=10:
        strdate = f"{date.year}-0{date.month}-{date.day}"
    else:
        strdate = f"{date.year}-0{date.month}-0{date.day}"

    for lon, lat, value in data[data.date == strdate].filter(['X','Y', iso_widget]).drop_duplicates().values:
        sensor_longitudes.append(lon)
        sensor_latitudes.append(lat)
        iso_values.append(float(value))
        

    
    fig = plt.figure(figsize=(12,9))

    # world map has a lower left corner of (-180, -90) and an upper right corner of (180, 90)
    m = Basemap(projection='mill',
                llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
                llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
                urcrnrlon=UPPER_LEFT_X_LON,
                urcrnrlat=UPPER_LEFT_Y_LAT,
                resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

    m.shadedrelief() # makes it look pretty :)

    # Locate Chernobyl
    m.scatter(CHERNOBYL_COORDS[0], CHERNOBYL_COORDS[1], latlon=True, s=100, c='green', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

    #Locate Each Sensor:
    sc = m.scatter(sensor_longitudes, sensor_latitudes, latlon=True, s=100, c=iso_values, cmap='hot', marker='o', alpha=1, vmin=colorbar[iso_widget]['min'], vmax=colorbar[iso_widget]['max']) 
    
    fig.colorbar(sc)
    plt.title(f"{date.month}-{date.day}-1986       {iso_widget}")
    if CREATE_PICS: 
        if iso_widget == "I_131_(Bq/m3)":
            fig.savefig(f'img/I131/{date.month}-{date.day}_I131.jpg',bbox_inches='tight',transparent=True, pad_inches=0)
        if iso_widget == "Cs_134_(Bq/m3)":
            fig.savefig(f'img/Cs134/{date.month}-{date.day}_Cs134.jpg',bbox_inches='tight',transparent=True, pad_inches=0)
        if iso_widget == "Cs_137_(Bq/m3)":
            fig.savefig(f'img/Cs137/{date.month}-{date.day}_Cs137.jpg',bbox_inches='tight',transparent=True, pad_inches=0)

        plt.close()
    else: plt.show()
    

In [ ]:
date_widget = widgets.DatePicker()
date_widget.value = datetime.date(1986, 4, 27)

iso_widget = widgets.Dropdown(
    options=['I_131_(Bq/m3)', 'Cs_134_(Bq/m3)', 'Cs_137_(Bq/m3)'],
    value='I_131_(Bq/m3)',
    description='Isotope:',
    disabled=False
)
ui = widgets.VBox([widgets.HBox([date_widget, iso_widget])])
out = widgets.interactive_output(create_map, {'date': date_widget, 'iso_widget': iso_widget})
display(ui, out)

# Interactive Map: Transparency

In [ ]:
def create_map_transparency(date, iso_widget, CREATE_PICS=False):
    sensor_latitudes = []
    sensor_longitudes = []
    iso_values = []


    if date.day >=10:
        strdate = f"{date.year}-0{date.month}-{date.day}"
    else:
        strdate = f"{date.year}-0{date.month}-0{date.day}"

    for lon, lat, value in data[data.date == strdate].filter(['X','Y', iso_widget]).drop_duplicates().values:
        sensor_longitudes.append(lon)
        sensor_latitudes.append(lat)
        iso_values.append(float(value))
    fig = plt.figure(figsize=(12,9))

    # world map has a lower left corner of (-180, -90) and an upper right corner of (180, 90)
    m = Basemap(projection='mill',
                llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
                llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
                urcrnrlon=UPPER_LEFT_X_LON,
                urcrnrlat=UPPER_LEFT_Y_LAT,
                resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

    m.shadedrelief() # makes it look pretty :)

    # Locate Chernobyl
    m.scatter(CHERNOBYL_COORDS[0], CHERNOBYL_COORDS[1], latlon=True, s=100, c='green', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

    alph = np.array(iso_values)/colorbar[iso_widget]['max']
    #Locate Each Sensor:
    # sc = m.scatter(sensor_longitudes, sensor_latitudes, latlon=True, s=50, c=iso_values, cmap='hot', marker='o', alpha=1, vmin=colorbar[iso_widget]['min'], vmax=colorbar[iso_widget]['max']) 
    sc = m.scatter(sensor_longitudes, sensor_latitudes, latlon=True, s=50, c='red', marker='o', alpha=alph) 
    
    # fig.colorbar(sc)
    if CREATE_PICS: 
        plt.title(f"{date.month}-{date.day}-1986       {iso_widget}")
        if iso_widget == "I_131_(Bq/m3)":
            fig.savefig(f'img/I131-trans/{date.month}-{date.day}_I131.jpg',bbox_inches='tight',transparent=True, pad_inches=0)
        if iso_widget == "Cs_134_(Bq/m3)":
            fig.savefig(f'img/Cs134-trans/{date.month}-{date.day}_Cs134.jpg',bbox_inches='tight',transparent=True, pad_inches=0)
        if iso_widget == "Cs_137_(Bq/m3)":
            fig.savefig(f'img/Cs137-trans/{date.month}-{date.day}_Cs137.jpg',bbox_inches='tight',transparent=True, pad_inches=0)

        plt.close()
    else: plt.show()
    

In [ ]:
date_widget = widgets.DatePicker()
date_widget.value = datetime.date(1986, 4, 27)

iso_widget = widgets.Dropdown(
    options=['I_131_(Bq/m3)', 'Cs_134_(Bq/m3)', 'Cs_137_(Bq/m3)'],
    value='I_131_(Bq/m3)',
    description='Isotope:',
    disabled=False
)
ui = widgets.VBox([widgets.HBox([date_widget, iso_widget])])
out = widgets.interactive_output(create_map_transparency, {'date': date_widget, 'iso_widget': iso_widget})
display(ui, out)

# Make Images

In [ ]:
isotopes = ["I_131_(Bq/m3)", "Cs_134_(Bq/m3)","Cs_137_(Bq/m3)"]

numdays = (datetime.date(1986, 6, 1) - datetime.date(1986, 4, 27)).days
base = datetime.date(1986, 4, 27)
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]

# date_list = [datetime.date(1986, 4, 27)]

for isotope in isotopes:
    for date in date_list:
        create_map_transparency(date,isotope,CREATE_PICS=True)
        create_map(date,isotope,CREATE_PICS=True)

print("Done")

# Wind Arrows

In [ ]:
def generate_arrow(map, destlon, destlat, date):
    chernobylStartDate = datetime.date(1986, 4, 26)
    numDays = (date - chernobylStartDate).days

    dist_x = destlon - CHERNOBYL_COORDS[0]
    dist_y = destlat - CHERNOBYL_COORDS[1]

    arrow_coords = []

    start_x = CHERNOBYL_COORDS[0]
    start_y = CHERNOBYL_COORDS[1]

    for _ in range(numDays):
        next_x = start_x + (dist_x / numDays)
        next_y = start_y + (dist_y / numDays)
        x, y = map(start_x, start_y)
        nx, ny = map(next_x, next_y)
        dx = nx - x
        dy = ny - y
        arrow_coords.append([x, y, dx, dy])
        start_x = next_x
        start_y = next_y

    return arrow_coords


In [ ]:
def create_map_wind(displaydate, iso_widget, CREATE_PICS=False):
    fig = plt.figure(figsize=(20,20))
    m = Basemap(projection='mill',
                    llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
                    llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
                    urcrnrlon=UPPER_LEFT_X_LON,
                    urcrnrlat=UPPER_LEFT_Y_LAT,
                    resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

    m.shadedrelief() # makes it look pretty :)

    # Locate Chernobyl
    m.scatter(CHERNOBYL_COORDS[0], CHERNOBYL_COORDS[1], latlon=True, s=100, c='green', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

    arrows = []


    for lon, lat, date, value in data.filter(['X','Y', 'date', iso_widget]).values:
        if float(value) < 1.5:
            continue
        s_year, s_month, s_day = date.split('-')
        arrow_path = generate_arrow(m, float(lon), float(lat), datetime.date(int(s_year), int(s_month), int(s_day)))
        arrows.append(arrow_path)

    if iso_widget == "I_131_(Bq/m3)":
        c = 'yellow'
    elif iso_widget == 'Cs_134_(Bq/m3)':
        c = 'blue'
    else:
        c = 'red'

    numdaysFromStart = (displaydate - datetime.date(1986,4,27)).days
    for arrow in arrows:
        if len(arrow) > numdaysFromStart:
            plt.arrow(
            x = arrow[numdaysFromStart][0],
            y = arrow[numdaysFromStart][1],
            dx = arrow[numdaysFromStart][2],
            dy = arrow[numdaysFromStart][3],
            head_width=100000, 
            head_length=100000,
            facecolor=c,
            length_includes_head=True
                        )
            
    if CREATE_PICS: 
        plt.title(f"{displaydate.month}-{displaydate.day}-1986       {iso_widget}")
        if iso_widget == "I_131_(Bq/m3)":
            fig.savefig(f'img/I131-wind/{displaydate.month}-{displaydate.day}_I131.jpg',bbox_inches='tight',transparent=True, pad_inches=0)
        if iso_widget == "Cs_134_(Bq/m3)":
            fig.savefig(f'img/Cs134-wind/{displaydate.month}-{displaydate.day}_Cs134.jpg',bbox_inches='tight',transparent=True, pad_inches=0)
        if iso_widget == "Cs_137_(Bq/m3)":
            fig.savefig(f'img/Cs137-wind/{displaydate.month}-{displaydate.day}_Cs137.jpg',bbox_inches='tight',transparent=True, pad_inches=0)

        plt.close()
    else: plt.show()



In [ ]:
date_widget = widgets.DatePicker()
date_widget.value = datetime.date(1986, 4, 27)

iso_widget = widgets.Dropdown(
    options=['I_131_(Bq/m3)', 'Cs_134_(Bq/m3)', 'Cs_137_(Bq/m3)'],
    value='I_131_(Bq/m3)',
    description='Isotope:',
    disabled=False
)
ui = widgets.VBox([widgets.HBox([date_widget, iso_widget])])
out = widgets.interactive_output(create_map_wind, {'displaydate': date_widget, 'iso_widget': iso_widget})
display(ui, out)

In [ ]:
isotopes = ["I_131_(Bq/m3)", "Cs_134_(Bq/m3)","Cs_137_(Bq/m3)"]

numdays = (datetime.date(1986, 6, 1) - datetime.date(1986, 4, 27)).days
base = datetime.date(1986, 4, 27)
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]

# date_list = [datetime.date(1986, 4, 27)]

for isotope in isotopes:
    for date in date_list:
        create_map_wind(date,isotope,CREATE_PICS=True)

print("Done")